# Import Data

In [41]:
import pandas as pd

In [42]:
df = pd.read_csv('C:/Users/User/Documents/le wagon project/coronanet_release_allvars.csv', encoding='mac_roman')

In [43]:
for col in df.columns: 
    print(col) 

Unnamed: 0
record_id
policy_id
entry_type
correct_type
update_type
update_level
description
date_announced
date_start
date_end
country
ISO_A3
ISO_A2
init_country_level
domestic_policy
province
city
type
type_sub_cat
type_text
institution_status
target_country
target_geog_level
target_region
target_province
target_city
target_other
target_who_what
target_direction
travel_mechanism
compliance
enforcer
index_high_est
index_med_est
index_low_est
index_country_rank
link
date_updated
recorded_date
confirmed_cases
deaths
recovered
new_tests
total_tests
total_tests_per_thousand
tests_units
Rank_FP
Score_FP
state_IDC
muni_IDC
dispersive_IDC
constraining_IDC
inclusive_IDC
sfi_SFI
ti_cpi_TI
pop_WDI_PW
gdp_WDI_PW
gdppc_WDI_PW
growth_WDI_PW
lnpop_WDI_PW
lngdp_WDI_PW
lngdppc_WDI_PW
disap_FA
polpris_FA
latentmean_FA
transparencyindex_HR
EmigrantStock_EMS
v2x_polyarchy_VDEM


# keep the important columns

In [44]:
 def tristans_columns(self):
        df = self.copy()
        df['date_announced'] = pd.to_datetime(df['date_announced'])
        df['date_start'] = pd.to_datetime(df['date_start'])
        df['date_end'] = pd.to_datetime(df['date_end'])
        df['date_announced'].fillna(df['date_start'], inplace = True)
        df['date_start'].clip(lower=df['date_announced'], inplace=True)
        df_select = df[['policy_id', 'date_start', 'country',
           'type', 'type_sub_cat','description', 'ISO_A3','compliance']]
        df_select.drop_duplicates(subset=['policy_id'], inplace=True)
        return df_select

In [45]:
df= tristans_columns(df)

c:\users\user\.venvs\lewagon\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [46]:
for col in df.columns: 
    print(col) 

policy_id
date_start
country
type
type_sub_cat
description
ISO_A3
compliance


In [47]:
df.isnull().sum(axis = 0)

policy_id           0
date_start          0
country             0
type                0
type_sub_cat    10257
description         0
ISO_A3            133
compliance        125
dtype: int64

In [48]:
df = df.dropna()

In [49]:
df.isnull().sum(axis = 0)

policy_id       0
date_start      0
country         0
type            0
type_sub_cat    0
description     0
ISO_A3          0
compliance      0
dtype: int64

In [50]:
df

,policy_id,date_start,country,type,type_sub_cat,description,ISO_A3,compliance
4,6721435,2020-03-09,Afghanistan,Closure and Regulation of Schools,Preschool or childcare facilities (generally f...,"""All schools in Herat ñ including temporarily ...",AFG,Mandatory (Unspecified/Implied)
7,6018486,2020-03-11,Afghanistan,Closure and Regulation of Schools,Primary Schools (generally for children ages 1...,"In parts of northern Samangan province, school...",AFG,Mandatory (Unspecified/Implied)
10,3558044,2020-03-14,Afghanistan,Closure and Regulation of Schools,Preschool or childcare facilities (generally f...,"""On 14 March, the Government of Afghanistan an...",AFG,Mandatory (Unspecified/Implied)
18,2231982,2020-04-18,Afghanistan,Closure and Regulation of Schools,Primary Schools (generally for children ages 1...,"April 11, Afghanistan's ""President Ghani says ...",AFG,Mandatory (Unspecified/Implied)
21,1212064,2020-06-03,Afghanistan,Closure and Regulation of Schools,Preschool or childcare facilities (generally f...,"April 14 ""Afghanistan on Saturday announced th...",AFG,Mandatory (Unspecified/Implied)
...,...,...,...,...,...,...,...,...
45629,6143876,2020-10-02,Zimbabwe,Restriction and Regulation of Government Services,Election procedures (e.g. mail-in voting),"The Zimbabwean minister for health, who is als...",ZWE,Mandatory (Unspecified/Implied)
45630,2531192,2020-03-02,Zimbabwe,Restrictions of Mass Gatherings,Prison population reduced (e.g. early release ...,A presidential amnesty was announced by the Zi...,ZWE,Voluntary/Recommended but No Penalties
45631,2574281,2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,Postponement of an annually recurring event,"In Zimbabwe, the 40th independence anniversary...",ZWE,Mandatory (Unspecified/Implied)
45632,6689905,2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,Postponement of an annually recurring event,"In Zimbabwe, the Zimbabwe International Trade ...",ZWE,Mandatory (Unspecified/Implied)


# Tokenization and lemmatization

In [51]:
import numpy as np
import pandas as pd
import datetime
import pyreadr
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [52]:
def clean(x):
    x = x.lower()
    x = ''.join([letter for letter in x if not letter.isdigit()])
    for p in string.punctuation:
        x = x.replace(p, " ")
    stops = set(stopwords.words("english"))
    tokens = word_tokenize(x)
    tokens = ' '.join([token for token in tokens if token not in stops])
    return tokens

In [53]:
df['clean_description'] = df.description.apply(clean)

In [55]:
df['clean_description']

4        schools herat ñ including temporarily learning...
7        parts northern samangan province schools close...
10       march government afghanistan announced schools...
18       april afghanistan president ghani says schools...
21       april afghanistan saturday announced closure e...
                               ...                        
45629    zimbabwean minister health also vice president...
45630    presidential amnesty announced zimbabwe govern...
45631    zimbabwe th independence anniversary celebrati...
45632    zimbabwe zimbabwe international trade fair zit...
45639    zimbabwe minister information publicity announ...
Name: clean_description, Length: 14555, dtype: object

In [56]:
def lemma(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

In [57]:
df['clean_description'] = lemma(df['clean_description'])

In [58]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

df['clean_description'] = [stemmer.stem(word) for word in df['clean_description']]


In [59]:
df['clean_description']

4        schools herat ñ including temporarily learning...
7        parts northern samangan province schools close...
10       march government afghanistan announced schools...
18       april afghanistan president ghani says schools...
21       april afghanistan saturday announced closure e...
                               ...                        
45629    zimbabwean minister health also vice president...
45630    presidential amnesty announced zimbabwe govern...
45631    zimbabwe th independence anniversary celebrati...
45632    zimbabwe zimbabwe international trade fair zit...
45639    zimbabwe minister information publicity announ...
Name: clean_description, Length: 14555, dtype: object

# Now lets encode my subcat

In [60]:
df[['type_sub_cat']]

,type_sub_cat
4,Preschool or childcare facilities (generally f...
7,Primary Schools (generally for children ages 1...
10,Preschool or childcare facilities (generally f...
18,Primary Schools (generally for children ages 1...
21,Preschool or childcare facilities (generally f...
...,...
45629,Election procedures (e.g. mail-in voting)
45630,Prison population reduced (e.g. early release ...
45631,Postponement of an annually recurring event
45632,Postponement of an annually recurring event


In [61]:
temp = df['type_sub_cat'].value_counts()

In [62]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit_transform(list(pd.DataFrame(temp).reset_index()['index']))

array([ 14,  85,  39,  62,  46,  77,  83,  47,   1,  63,  42,  48,  49,
        35,  29,  21,  96,  26,  33,  11,  53,   3,   9, 108,  15,  66,
        91,  45,  54,  84,  18,  31,  90,  89,  10,  28,  30,  51,  94,
       102,  58, 104,  50,  52,  73,   6,  44,  86,   4,  97,  40,   2,
        23,  57,  93,  60,  87, 101,  25,  68,  61,  20,  55,  22,  67,
        17,  78,  75,  27,  13,  69,  65, 103,   5,  95,  64,  16,  56,
        74,  24,  80,  43,  36,  72, 100,  98,  12,   7,  34, 111, 112,
       109,  19,  71,  59,  79,   8,   0,  92, 110,  99,  41,  88,  82,
        81, 105, 107,  37,  70,  76, 106,  38, 113,  32], dtype=int64)

In [63]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['type_sub_cat'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))


In [64]:
df['encoded_sub-cat']= df['type_sub_cat'].map(le_name_mapping)

# Now we can run a Naive Bayes model and see its score

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df = 0.05,max_features = 1000)
X = vectorizer.fit_transform(df['clean_description']) 
y = df['encoded_sub-cat']

In [66]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()

nb_model.fit(X,y)

nb_model.score(X,y)

0.4492614221916867

# lets double check to see if the model can predict the right sub-cat

In [67]:
nb_model.predict_proba(X)

array([[5.94573483e-04, 8.42802270e-03, 2.99009430e-03, ...,
        8.46898764e-04, 9.88512874e-04, 5.53636928e-05],
       [7.67306411e-04, 5.68897027e-02, 2.41427938e-03, ...,
        8.41616745e-04, 8.53155435e-04, 4.98004106e-05],
       [4.00406293e-04, 1.64920649e-02, 2.16785908e-03, ...,
        6.35062782e-04, 7.93507870e-04, 3.53953772e-05],
       ...,
       [3.92220827e-04, 1.95670411e-02, 6.25330530e-03, ...,
        6.51980239e-04, 7.78684936e-04, 4.23748579e-05],
       [4.36394755e-04, 1.77963930e-02, 7.08033155e-03, ...,
        7.86107728e-04, 9.29079831e-04, 4.78307787e-05],
       [5.77005705e-04, 3.46218597e-02, 8.27846311e-03, ...,
        9.04338908e-04, 8.14233062e-04, 5.46335257e-05]])

## I'll take the 46th line 

In [68]:
sorted(nb_model.predict_proba(X)[15], reverse = True)[0:3]

[0.22193185495054904, 0.14100540760585478, 0.04458132248781279]

In [69]:
nb_model.predict_proba(X)[15].argmax()

63

In [70]:
df.iloc[5]

policy_id                                                      1939072
date_start                                         2020-08-05 00:00:00
country                                                    Afghanistan
type                                 Closure and Regulation of Schools
type_sub_cat         Higher education institutions (i.e. degree gra...
description          Aug 4, Afghanistan "has announced that all pri...
ISO_A3                                                             AFG
compliance                      Voluntary/Recommended but No Penalties
clean_description    aug afghanistan announced private public unive...
encoded_sub-cat                                                     28
Name: 36, dtype: object

In [71]:
nb_model.predict_proba(X)[15][63]

0.22193185495054904

In [72]:
le.classes_[63]

'Primary Schools (generally for children ages 10 and below)'

## Well... got it wrong this time, but he gets it right for the first one (try urself)

How do I increase my score ? Need to do a nice grid search, maybe find a way to regroup some of the sub-cats, maybe only take the top 3 results, then REDO a naive Bayse pred from those => OVER THE WEEKEND

## Gonna try with a pipeline and Gridsearch now

In [73]:
X =(df['clean_description']) 
y = df['encoded_sub-cat']

In [74]:
df.dtypes

policy_id                     int64
date_start           datetime64[ns]
country                      object
type                         object
type_sub_cat                 object
description                  object
ISO_A3                       object
compliance                   object
clean_description            object
encoded_sub-cat               int64
dtype: object

In [79]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB()),
])

# Set parameters to search
parameters = {
    'tfidf__ngram_range': ((1,1),(2,2)),
    'tfidf__max_features': (1000,2000,3000),
    'tfidf__max_df': (1,0.9,0.8,0.7),
    'nb__alpha': (0,0.1)}

# Perform grid search
grid_search = GridSearchCV(pipeline, parameters, n_jobs=1, 
                           verbose=1, scoring = "accuracy", 
                           refit=True, cv=5)




In [80]:
#grid_search.fit(X,y)

In [81]:
grid_search.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [78]:
grid_search.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

After multiple gridsearches, I realised this wasnt working so I stuck to this one for now (I'm testing it on the last 155 subcats here):

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df = 0.05,max_features = 1000)
X = vectorizer.fit_transform(df['clean_description'])[:14400]
y = df['encoded_sub-cat'][:14400]

In [83]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()

nb_model.fit(X,y)

nb_model.score(X,y)

0.44979166666666665

In [84]:
A = nb_model.predict(vectorizer.fit_transform(df['clean_description'])[14400:])

In [85]:
B =df['encoded_sub-cat'][14400:]

In [86]:
C = A==B

In [87]:
C.value_counts()

False    108
True      47
Name: encoded_sub-cat, dtype: int64

## 59 true / 155 (keep that in mind for later)

# Now im gonna try a SGDC model to see if it can give me a better pred

In [88]:
from sklearn.linear_model import SGDClassifier

In [89]:
SGDC = SGDClassifier()
#No params for now
# loss =‘hinge’, ‘log’, ‘modified_huber’, ‘squared_hinge’, ‘perceptron’, or a regression loss: ‘squared_loss’, ‘huber’, ‘epsilon_insensitive’, or ‘squared_epsilon_insensitive’

In [90]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_description'])[:14400]
y = df['encoded_sub-cat'][:14400]

In [91]:
SGDC.fit(X,y)

c:\users\user\.venvs\lewagon\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [92]:
SGDC.score(X,y)

0.9179166666666667

### That seems waaaaay too high to be possible, i'll try it in case

In [93]:
A = SGDC.predict(vectorizer.fit_transform(df['clean_description'])[14400:])

In [94]:
B =df['encoded_sub-cat'][14400:]

In [99]:
C = A==B

In [100]:
C.value_counts()

False    94
True     61
Name: encoded_sub-cat, dtype: int64

In [101]:
65/155

0.41935483870967744

## 65 true / 155 Already better !

## Now i need to play with the params to get the best pred, this might take some time...

In [102]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('SGDC', SGDClassifier()),
    'nb', MultinomialNB()
])

# Set parameters to search
parameters = {
    'tfidf__max_features': (1000,2000, 5000,10000),
    'tfidf__max_df': (1,0.8,0.6),
    'SGDC__loss': ('log', 'modified_huber', 'squared_hinge', 'perceptron'),
    'SGDC__alpha': (0.0001, 0.001, 0.01, 0.1, 1)}

# Perform grid search
grid_search = GridSearchCV(pipeline, parameters, n_jobs=1, 
                           verbose=5600, scoring = "accuracy", 
                           refit=True, cv=5)

TypeError: zip argument #4 must support iteration

In [64]:
X =(df['clean_description']) 
y = df['encoded_sub-cat']

In [176]:
#grid_search.fit(X,y)

In [67]:
grid_search.best_score_grid_search.best_params_

0.44706286499484715

In [68]:
grid_search.best_params_

{'SGDC__alpha': 0.001,
 'SGDC__loss': 'squared_hinge',
 'tfidf__max_df': 0.2,
 'tfidf__max_features': 5000,
 'tfidf__ngram_range': (1, 1)}

In [164]:
vectorizer = TfidfVectorizer(max_df= 0.2, max_features= 5000, ngram_range= (1, 1))
X = vectorizer.fit_transform(df['clean_description'])[:14400]
y = df['encoded_sub-cat'][:14400]

In [165]:
SGDC = SGDClassifier(alpha= 0.001, loss= 'squared_hinge')
SGDC.fit(X,y)

c:\users\user\.venvs\lewagon\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='squared_hinge',
       max_iter=None, n_iter=None, n_iter_no_change=5, n_jobs=None,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       tol=None, validation_fraction=0.1, verbose=0, warm_start=False)

In [173]:
SGDC.score(X,y)

0.7127777777777777

In [177]:
A = SGDC.predict(vectorizer.fit_transform(df['clean_description'])[14400:])
B =df['encoded_sub-cat'][14400:]
C = A==B
C.value_counts()

False    92
True     63
Name: encoded_sub-cat, dtype: int64

In [179]:
(63/155)*100

40.64516129032258

## Guess I cant get higher than this..

## Lets try some new models

In [125]:
import xgboost as xg
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, encoding="utf-8",  
      ngram_range = (1,3),  
      token_pattern = "[A-Za-z_][A-Za-z\d_]*")
xgb_r = xg.XGBClassifier()

In [127]:
X = cv.fit_transform(df.clean_description).toarray()
y = df[['encoded_sub-cat']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)



In [ ]:
xgb_r.fit(X_train,y_train)

In [121]:
xgb_r.score(X_test,y_test)

c:\users\user\.venvs\lewagon\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\user\.venvs\lewagon\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields clean_description

In [105]:
xgb_r.score(X,y)

0.5560427968606463